In [1]:
module StonerWohlfarthModel 

using Optim, StaticArrays, Plots


function gr!(storage, x, h::Float64, p::MVector)
    storage[1] = -0.5*sin(psi-x[1])+h*sin(x[1]);
end

function apply_field!(ψ, ϕ, h)
    energy(x) = 0.5*sin(ψ-x[1])^2- h*cos.(x[1]);
    gradient(storage, x) = gr!(storage, x, h, p);
    #optimization_result = optimize(energy, [ϕ], Newton());
    optimization_result = optimize(energy,[ϕ], BFGS());
    return Optim.minimizer(optimization_result)[1];
end

function draw_representation(ψ::Float64)
    hstep = 0.01
    hmax = 2.0
    h = [collect(hmax:-hstep:-hmax); collect(-hmax:hstep:hmax)]
    len = length(h)
    println(len)
    m = zeros(len)
    
    ϕ = ψ
    for i=1:len
        ϕ = apply_field!(ψ, ϕ, h[i])
        m[i] = cos(ϕ)
    end
    plot(h, m, xlim = (-2.2, 2.2), ylim = (-1.1,1.1), aspect_ratio=[1 2], legend=:none, xlabel="Field", ylabel = "Magnetization");
end

function calculate_particles(psis::Array{Float64}, h::Array{Float64})
    n = length(psis)
    len = length(h)
    phis = copy(psis)
    
    m = zeros(n, len)
    for p = 1:n
        for i = 1:len
            phis[p] = apply_field!(psis[p], phis[p], h[i])
            m[p,i] = cos(phis[p])
        end
    end
    return m
end

export draw_representation, calculate_particles
end

StonerWohlfarthModel

In [2]:
module RandomSpherePoints

function get_points_spherical_random(n::Int)
    r = ones(n)
    θ = @. acos(1-2rand(n))
    ϕ = @. 2pi*rand(n)
    
    return [r θ ϕ]
end

function get_points_spherical_evenly(n::Int)
    r = ones(n)
    θ = @. acos(1-2linspace(0.0, 1.0, n))
    ϕ = @. 2pi*linspace(0.0, 1.0, n)
    
    return [r θ ϕ]
end

function get_points_cartesian_random(n::Int)
    points = get_points_spherical_random(n)
    
    return [sin(points[:,2]).*cos(points[:,3]) sin(points[:,2]).*sin(points[:,3]) cos(points[:,2])]
end

function random_sphere_pack(r::AbstractFloat, R::AbstractFloat, n::Int)
        
    step = r/10
    x = [ collect(-step : -step : -R); collect(0 : step : R)]
    y = [ collect(-step : -step : -R); collect(0 : step : R)]
    z = [ collect(-step : -step : -R); collect(0 : step : R)]
    len = length(x)
    places = zeros(len, len, len)
    
    for i = 1:len
        for j = 1:len
            for k = 1:len
                if sqrt(x[i]^2 + y[j]^2 + z[k]^2) < R
                    places[i,j,k] = 1
                end
            end
        end
    end
    
    points = zeros(n, 3)
    count = 0
    
    while count < n
        vacant_points = find(places)
        if isempty(vacant_points)
            error("there are no more vacant points in the space of the cluster at `count`=$count")
        end
        
        random_vacant_point = vacant_points[rand(1:length(vacant_points))]
        (I,J,K) = ind2sub(places, random_vacant_point)
        
        if places[I,J,K] != 1
            display("Something went wrong! places(p(1),p(2),p(3))!=1")
        end
        
        for i = 1:len
            for j = 1:len
                for k = 1:len
                    if sqrt((x[i]-x[I])^2 + (y[j]-y[J])^2 + (z[k]-z[K])^2) < 2r
                        places[i,j,k] = 0
                    end
                end
            end
        end
        
        count += 1;
        points[count,:] = [x[I] y[J] z[K]]
    end
    
    return points
end

export get_points_spherical_random
export get_points_cartesian_random
export get_points_spherical_evenly
export random_sphere_pack

end

RandomSpherePoints

In [12]:
using StaticArrays, StonerWohlfarthModel, BenchmarkTools, Plots, RandomSpherePoints
N = 1000
directions = get_points_spherical_random(N)
psis = directions[:,2]
hstep = 0.01
hmax = 2.0
h = [collect(0:hstep:hmax); collect(hmax:-hstep:-hmax); collect(-hmax:hstep:hmax)]
#display(@btime m=calculate_particles(psis, h))
m=calculate_particles(psis, h)
plot(h,sum(m,1)'/N, xlim=(-2.2, 2.2), ylim = (-1.1, 1.1), aspect_ratio=[1 2], legend=:none, xlabel="Field", ylabel = "Magnetization")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 Field 
 
 
 Magnetization 
 
<polyline clip-path="url(#clip7902)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 317.783,186.281 318.976,185.49 320.169,184.699 321.362,183.908 322.556,183.116 323.749,182.324 324.942,181.53 326.136,180.735 327.329,179.939 328.522,179.142 
 329.716,178.342 330.909,177.541 332.102,176.738 333.296,175.932 334.489,175.123 335.682,174.312 336.876,173.497 338.069,172.679 339.262,171.857 340.456,171.031 
 341.649,170.201 342.842,169.366 344.036,168.526 345.229,167.681 346.422,166.829 347.616,165.972 348.809,165.108 350.002,164.236 351.196,163.357 352.389,162.469 
 353.582,161.572 354.776,160.666 355.969,159.748 357.162,158.82 358.356,157.878 359.549,156.923 360.742,155.952 361.936,154.965 363.129,153.96 364.322,152.934 
 365.516,151.885 366.709,150.809 367.902,149.704 369.096,148.564 370.289,147.384 371.482,146.154 372.676,144.862 373.869,143.491 375.062,142.008 376.256,140.347 
 377.449,130.874 378.642,122.786 379.836,118.274 381.029,113.121 382.222,108.96 383.416,106.153 384.609,103.203 385.802,100.875 386.996,98.5586 388.189,96.4696 
 389.382,95.1507 390.576,94.1611 391.769,92.8577 392.962,91.8184 394.156,90.8273 395.349,90.2563 396.542,89.5343 397.736,88.8394 398.929,88.3676 400.122,87.9176 
 401.316,87.2636 402.509,86.8432 403.702,86.4444 404.896,86.0391 406.089,85.4562 407.282,85.0577 408.476,84.7099 409.669,84.3533 410.862,83.99 412.056,83.6694 
 413.249,83.1142 414.442,82.5256 415.636,82.2019 416.829,81.6717 418.022,81.3538 419.215,81.0578 420.409,80.7522 421.602,80.4538 422.795,80.1689 423.989,79.9325 
 425.182,79.6559 426.375,79.3782 427.569,79.0976 428.762,78.8217 429.955,78.5582 431.149,78.2941 432.342,78.0353 433.535,77.7821 434.729,77.5345 435.922,77.2925 
 437.115,77.0568 438.309,76.829 439.502,76.6077 440.695,76.3926 441.889,76.1837 443.082,75.9808 444.275,75.7839 445.469,75.5927 446.662,75.4071 447.855,75.2271 
 449.049,75.0523 450.242,74.8827 451.435,74.718 452.629,74.5582 453.822,74.4029 455.015,74.2522 456.209,74.1057 457.402,73.9633 458.595,73.825 459.789,73.6904 
 460.982,73.5596 462.175,73.4324 463.369,73.3086 464.562,73.1881 465.755,73.0709 466.949,72.9567 468.142,72.8456 469.335,72.7373 470.529,72.6318 471.722,72.529 
 472.915,72.4288 474.109,72.3312 475.302,72.2359 476.495,72.1431 477.689,72.0525 478.882,71.9641 480.075,71.8779 481.269,71.7937 482.462,71.7115 483.655,71.6313 
 484.849,71.5529 486.042,71.4764 487.235,71.4016 488.429,71.3285 489.622,71.2571 490.815,71.1874 492.009,71.1191 493.202,71.0524 494.395,70.9872 495.589,70.9234 
 496.782,70.861 497.975,70.7999 499.169,70.7401 500.362,70.6816 501.555,70.6244 502.749,70.5683 503.942,70.5135 505.135,70.4597 506.329,70.4071 507.522,70.3555 
 508.715,70.305 509.909,70.2555 511.102,70.207 512.295,70.1595 513.489,70.1129 514.682,70.0672 515.875,70.0224 517.069,69.9784 518.262,69.9354 519.455,69.8931 
 520.648,69.8516 521.842,69.811 523.035,69.7711 524.228,69.7319 525.422,69.6935 526.615,69.6557 527.808,69.6187 529.002,69.5823 530.195,69.5466 531.388,69.5116 
 532.582,69.4771 533.775,69.4433 534.968,69.4101 536.162,69.3775 537.355,69.3454 538.548,69.3139 539.742,69.2829 540.935,69.2525 542.128,69.2226 543.322,69.1932 
 544.515,69.1643 545.708,69.1358 546.902,69.1079 548.095,69.0804 549.288,69.0534 550.482,69.0268 551.675,69.0006 552.868,68.9749 554.062,68.9496 555.255,68.9247 
 556.448,68.9002 556.448,68.9002 555.255,68.9247 554.062,68.9496 552.868,68.9749 551.675,69.0006 550.482,69.0268 549.288,69.0534 548.095,69.0804 546.902,69.1079 
 545.708,69.1358 544.515,69.1643 543.322,69.1932 542.128,69.2226 540.935,69.2525 539.742,69.2829 538.548,69.3139 537.355,69.3454 536.162,69.3775 534.968,69.4101 
 533.775,69.4433 532.582,69.4771 531.388,69.5116 530.195,69.5466 529.002,69.5823 527.808,69.

In [4]:
plot(h,sum(m,1)'/N, xlim=(-2.2, 2.2), ylim = (-1.1, 1.1), aspect_ratio=[1 2], legend=:none, xlabel="Field", ylabel = "Magnetization")
savefig("./SwParticleDistribution.jpg")

In [ ]:
using RandomSpherePoints
points = random_sphere_pack(30.0, 300.0, 30)

In [6]:
scatter(points[:,1],points[:,2],points[:,3], linestyle = :dot, markersize = 20)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7600)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 129.552,354.363 129.552,354.363 126.887,355.653 129.552,354.363 129.552,319.608 128.22,320.254 129.552,319.608 129.552,284.854 126.887,286.145 129.552,284.854 
 129.552,250.1 128.22,250.745 129.552,250.1 129.552,215.346 126.887,216.636 129.552,215.346 129.552,180.592 128.22,181.237 129.552,180.592 129.552,145.837 
 126.887,147.128 129.552,145.837 129.552,111.083 128.22,111.728 129.552,111.083 129.552,76.329 126.887,77.6193 129.552,76.329 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [ ]:
anim = @animate for i=1:N
           plot(h,m[i,:], xlim=(-2.2, 2.2), ylim = (-1.1, 1.1), aspect_ratio=[1 2], legend = :none, xlabel="Field", ylabel = "Magnetization")
        end
gif(anim, "./anim_sw_particles.gif", fps = 10) 

In [ ]:
for i=1:N
           display(plot(h,m[i,:], xlim=(-2.2, 2.2), ylim = (-1.1, 1.1), aspect_ratio=[1 2], title = string(round(180*psis[i]/pi,2))))
        end

In [13]:
using StonerWohlfarthModel
ψ=60*pi/180
draw_representation(ψ)

802


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 Field 
 
 
 Magnetization 
 
<polyline clip-path="url(#clip8102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 556.448,70.4209 555.255,70.4601 554.062,70.4998 552.868,70.54 551.675,70.5809 550.482,70.6223 549.288,70.6644 548.095,70.707 546.902,70.7503 545.708,70.7942 
 544.515,70.8387 543.322,70.8839 542.128,70.9298 540.935,70.9763 539.742,71.0235 538.548,71.0714 537.355,71.12 536.162,71.1694 534.968,71.2195 533.775,71.2703 
 532.582,71.3219 531.388,71.3743 530.195,71.4275 529.002,71.4815 527.808,71.5363 526.615,71.592 525.422,71.6484 524.228,71.7058 523.035,71.764 521.842,71.8232 
 520.648,71.8832 519.455,71.9442 518.262,72.0062 517.069,72.0691 515.875,72.1329 514.682,72.1978 513.489,72.2637 512.295,72.3306 511.102,72.3986 509.909,72.4677 
 508.715,72.5378 507.522,72.6091 506.329,72.6815 505.135,72.7551 503.942,72.8298 502.749,72.9058 501.555,72.9829 500.362,73.0613 499.169,73.141 497.975,73.222 
 496.782,73.3042 495.589,73.3878 494.395,73.4728 493.202,73.5591 492.009,73.6469 490.815,73.7361 489.622,73.8267 488.429,73.9189 487.235,74.0126 486.042,74.1078 
 484.849,74.2046 483.655,74.3029 482.462,74.403 481.269,74.5046 480.075,74.608 478.882,74.7131 477.689,74.8199 476.495,74.9286 475.302,75.039 474.109,75.1513 
 472.915,75.2655 471.722,75.3816 470.529,75.4996 469.335,75.6196 468.142,75.7417 466.949,75.8658 465.755,75.992 464.562,76.1203 463.369,76.2508 462.175,76.3836 
 460.982,76.5185 459.789,76.6558 458.595,76.7954 457.402,76.9373 456.209,77.0817 455.015,77.2285 453.822,77.3778 452.629,77.5297 451.435,77.6841 450.242,77.8412 
 449.049,78.0009 447.855,78.1634 446.662,78.3286 445.469,78.4966 444.275,78.6675 443.082,78.8413 441.889,79.0181 440.695,79.1978 439.502,79.3807 438.309,79.5666 
 437.115,79.7557 435.922,79.948 434.729,80.1435 433.535,80.3424 432.342,80.5447 431.149,80.7504 429.955,80.9595 428.762,81.1722 427.569,81.3885 426.375,81.6085 
 425.182,81.8321 423.989,82.0596 422.795,82.2908 421.602,82.5259 420.409,82.765 419.215,83.0081 418.022,83.2552 416.829,83.5065 415.636,83.762 414.442,84.0217 
 413.249,84.2857 412.056,84.5541 410.862,84.827 409.669,85.1044 408.476,85.3863 407.282,85.6729 406.089,85.9642 404.896,86.2603 403.702,86.5612 402.509,86.867 
 401.316,87.1778 400.122,87.4937 398.929,87.8147 397.736,88.1409 396.542,88.4723 395.349,88.8091 394.156,89.1513 392.962,89.4989 391.769,89.8521 390.576,90.211 
 389.382,90.5755 388.189,90.9458 386.996,91.322 385.802,91.7041 384.609,92.0922 383.416,92.4864 382.222,92.8868 381.029,93.2934 379.836,93.7063 378.642,94.1257 
 377.449,94.5515 376.256,94.9839 375.062,95.4229 373.869,95.8687 372.676,96.3213 371.482,96.7808 370.289,97.2473 369.096,97.7209 367.902,98.2016 366.709,98.6896 
 365.516,99.185 364.322,99.6877 363.129,100.198 361.936,100.716 360.742,101.242 359.549,101.775 358.356,102.317 357.162,102.866 355.969,103.424 354.776,103.989 
 353.582,104.564 352.389,105.146 351.196,105.737 350.002,106.337 348.809,106.946 347.616,107.563 346.422,108.189 345.229,108.825 344.036,109.47 342.842,110.124 
 341.649,110.788 340.456,111.461 339.262,112.145 338.069,112.838 336.876,113.541 335.682,114.254 334.489,114.978 333.296,115.712 332.102,116.457 330.909,117.213 
 329.716,117.98 328.522,118.758 327.329,119.548 326.136,120.349 324.942,121.162 323.749,121.987 322.556,122.824 321.362,123.674 320.169,124.536 318.976,125.411 
 317.783,126.299 316.589,127.201 315.396,128.117 314.203,129.046 313.009,129.99 311.816,130.948 310.623,131.921 309.429,132.91 308.236,133.914 307.043,134.934 
 305.849,135.971 304.656,137.025 303.463,138.095 302.269,139.184 301.076,140.291 299.883,141.417 298.689,142.562 297.496,143.728 296.303,144.914 295.109,146.121 
 293.916,147.351 292.723,148.603 291.529,149.88 290.336,151.181 289.143,152.508 287.949,153.863 286.75

In [10]:
using Plots
savefig("./SingleParticle")